### <center>2018 Winter CS101.06</center>

# <center>栈和队列</center>

##### <center>by tanzhuxiaqiu@huawei.com</center>

## 作业-03

- 2/12 19:00PM
- 简单讲解

## 今日议程

1. 栈
2. 队列
3. 双端队列

## 栈

遵循后进先出(LIFO)规则的对象集合。

常见的实例：
- 虚拟内存的栈空间
- 编译器中处理运算
- 浏览器的历史访问
- 编辑器的撤销

![](img/6-1.png)

### 栈的抽象数据类型（ADT）

如果用S表示一个栈的实例，S必须支持两个基本操作：

- S.push(e): 将一个元素e压入栈S的栈顶。
- S.pop()：如果栈S中至少含有一个元素，从栈S的栈顶弹出一个元素并返回其值；如果栈S为空，则抛出异常。

此外，为了方便操作，栈S还应支持：

- S.peek()：返回栈顶的元素值，但并不弹出该元素；如果栈S为空，则抛出异常。
- S.is_empty(): 判断栈S是否为空，如果为空就返回True；反之返回False。
- len(S): 返回栈S中元素的个数。
- iter(S): 返回一个Iterator。

In [1]:
from abc import ABC, abstractmethod


class AbstractStack(ABC):
    """Abstract Class for Stacks."""
    def __init__(self):
        self._top = -1

    def __len__(self):
        return self._top + 1

    def __repr__(self):
        res = "->".join(map(str, self))
        return 'Top->' + res

    def is_empty(self):
        return self._top == -1

    @abstractmethod
    def __iter__(self):
        raise NotImplementedError

    @abstractmethod
    def push(self, e):
        raise NotImplementedError

    @abstractmethod
    def pop(self):
        raise NotImplementedError

    @abstractmethod
    def peek(self):
        raise NotImplementedError


### 基于Python的List实现栈

|栈方法|对应List实现|
|---|---|
|S.push(e)|L.append(e)|
|S.pop()|L.pop()|
|S.peek()|L[-1]|
|S.is_empty|len(L) == 0|
|len(S)|len(L)|

In [2]:
class ArrayStack(AbstractStack):
    """LIFO Stack implementation using a Python list as underlying storage.
    """

    def __init__(self):
        """Create an empty stack."""
        super().__init__()
        self._data = []

    def __iter__(self):
        p = self._top
        while p >= 0:
            yield self._data[p]
            p -= 1

    def push(self, e):
        """Add element e to the top of stack."""
        self._top += 1
        self._data.append(e)

    def pop(self):
        """Remove and return the element from the top of the stack.
        Raise Empty exception if the stack is empty.
        """
        if self.is_empty():
            raise IndexError("Stack is empty")
        self._top -= 1
        return self._data.pop()

    def peek(self):
        """Return (not remove) the element at the top of the stack.
        Raise Empty exception if the stack is empty.
        """
        if self.is_empty():
            raise IndexError("Stack is empty")
        return self._data[self._top]


In [3]:
S = ArrayStack()
S.is_empty()

True

In [4]:
S.push(3)
S.push('a')
S.push({'k1': 'v1', 'k2': 'v2'})
S

Top->{'k1': 'v1', 'k2': 'v2'}->a->3

In [5]:
S.pop()
S

Top->a->3

In [6]:
len(S)

2

In [7]:
S.peek()

'a'

In [8]:
S._data

[3, 'a']

### 基于List的栈的时间复杂度

|操作|示例|时间复杂度|注释|
|---|---|---|---|
| Push | S.push(e) | O(1)* |均摊时间复杂度|
| Pop | S.pop() | O(1)* |均摊时间复杂度|
| Peek | S.peek() | O(1) | |
| Is empty | S.is_empty() | O(1) | |
| Length | len(S) | O(1) | |

In [9]:
def delimiter_matched(expr):
    """Return True if all delimiters are properly match; False otherwise.
    
    >>> delimiter_matched('[(2+x)*(3+y)]')
    True
    >>> delimiter_matched('{[{(xbcd))]}')
    False
    """
    left, right = '({[', ')}]'
    S = ArrayStack()

    for c in expr:
        if c in left:
            S.push(c)
        elif c in right:
            if S.is_empty():
                return False
            if right.index(c) != left.index(S.pop()):
                return False
    return S.is_empty()

In [10]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=2)

## 队列

与栈相似，但队列是遵循先进先出(FIFO)规则的对象集合。

- 常用于优化资源请求
- 插入元素的一端为队列尾部
- 删除元素的一端为队列头部


![](img/6-2.png)

### 队列的抽象数据类型（ADT）

如果用Q表示一个队列的实例，Q应该支持两个基本操作：

- Q.enqueue(e): 在队列Q的尾部添加一个元素e。
- Q.dequeue()：如果队列Q中至少含有一个元素，从队列Q的头部移除一个元素并返回其值；如果队列Q为空，则抛出异常。

此外，为了方便操作，栈S还应支持：

- Q.peek()：在不移除队列Q头部元素的情况下返回元素的值；如果队列Q为空，则抛出异常。
- Q.is_empty(): 判断队列Q是否为空，如果为空就返回True；反之返回False。
- len(Q): 返回队列Q中元素的个数。
- iter(Q): 返回一个Iterator。

In [ ]:
from abc import ABC, abstractmethod


class AbstractQueue(ABC):

    def __init__(self):
        self._size = 0

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    @abstractmethod
    def enqueue(self, e):
        raise NotImplementedError

    @abstractmethod
    def dequeue(self):
        raise NotImplementedError

    @abstractmethod
    def peek(self):
        raise NotImplementedError

    @abstractmethod
    def __iter__(self):
        raise NotImplementedError


### 基于Python的List实现队列

#### 思考

在实现栈时我们使用了一个标识位（指针）top来标示栈顶的元素位置，如果要实现一个队列需要几个标识位（指针）？还需要什么额外的算法逻辑吗？



#### 头尾都用标识位的线性队列

- 在队列头部第一个元素处设置一个标识符（头指针）front，当完成出列操作dequeue后此标识符相应后移一位
- 在队列尾部最后一个元素的后一位设置一个标识符（尾指针）rear，当完成入列操作enqueue后此标识符后移一位
![](img/6-3.png)

In [ ]:
class ArrayQueue(AbstractQueue):
    def __init__(self, cap=10):
        super().__init__()
        self._array = [None] * cap
        self._front = 0
        self._rear = 0

    def __iter__(self):
        p = self._front
        while p <= self._rear:
            yield self._array[p]
            p += 1

    def enqueue(self, e):
        if self._rear == len(self._array):
            self._expand()
        self._array[self._rear] = e
        self._rear += 1
        self._size += 1

    def _expand(self):
        """expands cap of the array.
        Time Complexity: O(n).
        """
        self._array += [None] * len(self._array)

    def dequeue(self):
        if self.is_empty():
            raise IndexError("Queue is empty")
        res = self._array[self._front]
        self._array[self._front] = None
        self._front += 1
        self._size -= 1
        return res

    def peek(self):
        if self.is_empty():
            raise IndexError("Queue is empty")
        return self._array[self._front]


In [ ]:
queue = ArrayQueue()
for i in range(15):
    queue.enqueue(i)

for i in range(10):
    queue.dequeue()
    
queue.enqueue('a')
queue.enqueue('b')

queue._array

#### 线性队列的优缺点

- 双标识符代码便于理解，但是底层的List在dequeue后空间会造成浪费
- 如何更合理的利用空间？gc回收？利用List自带的pop(0)来实现dequeue？

#### 循环队列

- 将ArrayQueue的首尾相接形成闭环
- 在enqueue和dequeue操作后通过取模来定位标识符

![](img/6-4.png)

#### 循环队列的算法逻辑

1. 初始化循环队列时，根据输入的capacity参数设置底层的List大小，初始化头指针front和尾指针rear；
2. 入列：
    - 如果队列已满（尾指针已处于最后一个空位时），抛出异常
    - 如果队列还有空位就在尾指针除插入元素，并将尾指针后移一位
3. 出列：
    - 如果队列为空（头尾指针相遇时），抛出异常
    - 如果队列中还有元素则将头指针标记的元素删除且返回，并将头指针后移一位
4. 元素个数：
    - rear >= front, size = rear - front
    - front > rear, size = capacity - (front - rear)

In [ ]:
class CircularQueue:
    def __init__(self, cap=10):
        self._array = [None] * cap
        self._front = 0
        self._rear = 0

    def __len__(self):
        if self._rear >= self._front:
            return self._rear - self._front
        return len(self._array) - (self._front - self._rear)

    def enqueue(self, e):
        if len(self) == len(self._array) - 1:
            raise IndexError("Queue is full")
        self._array[self._rear] = e
        self._rear = (self._rear + 1) % len(self._array)

    def dequeue(self):
        if len(self) == 0:
            raise IndexError("Queue is empty")
        res = self._array[self._front]
        self._array[self._front] = None
        self._front = (self._front + 1) % len(self._array)
        return res

In [ ]:
queue = CircularQueue()
for i in range(9):
    queue.enqueue(i)
queue._array

In [ ]:
queue.dequeue()
queue.enqueue('a')
queue.dequeue()
queue.enqueue('b')
queue._array

#### 双标识符环形队列的优缺点：

- 可以循环利用已使用过的空间；
- 元素个数的计算比较复杂，而且尾指针标识的空间无法存储元素。
- 有没有更优化的方式实现环形队列？并让队列支持动态的扩容和缩容？


In [ ]:
class CircularQueue(AbstractQueue):
    """Circular Queue implementation using a Python list as underlying storage.
    """
    def __init__(self, cap=10):
        super().__init__()
        self._array = [None] * cap
        self._front = 0

    def __iter__(self):
        p = self._front
        while p != (self._front + self._size) % len(self._array):
            yield self._array[p]
            p = (p + 1) % len(self._array)

    def enqueue(self, e):
        if self._size == len(self._array):
            self._resize(2 * len(self._array))
        rear = (self._front + self._size) % len(self._array)
        self._array[rear] = e
        self._size += 1

    def _resize(self, cap):
        tmp = self._array
        self._array = [None] * cap
        p = self._front
        for i in range(self._size):
            self._array[i] = tmp[p]
            p = (p + 1) % len(tmp)
        self._front = 0

    def dequeue(self):
        if self.is_empty():
            raise IndexError("Queue is empty")
        res = self._array[self._front]
        self._array[self._front] = None
        self._front = (self._front + 1) % len(self._array)
        self._size -= 1
        if 0 < self._size < len(self._array) // 4:
            self._resize(len(self._array) // 2)
        return res

    def peek(self):
        if self.is_empty():
            raise IndexError("Queue is empty")
        return self._array[self._front]

In [ ]:
queue = CircularQueue()
for i in range(10):
    queue.enqueue(i)
print(queue._array)

In [ ]:
queue.dequeue()
queue.enqueue('a')
queue.dequeue()
queue.enqueue('b')
print(queue._array)

In [ ]:
queue.enqueue('c')
print(queue._array)

### 基于List的环形队列的时间复杂度

|操作|示例|时间复杂度|注释|
|---|---|---|---|
| Enqueue | Q.enqueue(e) | O(1)* |均摊时间复杂度|
| Dequeue | Q.dequeue() | O(1)* |均摊时间复杂度|
| Peek | S.peek() | O(1) | |
| Is empty | S.is_empty() | O(1) | |
| Length | len(S) | O(1) | |

## 双端队列

Double-ended queue(deque): 双向的FIFO队列，支持在头部和尾部都可以插入和删除元素。

![](img/6-5.png)

### 双端队列的抽象数据类型（ADT）

如果用D表示一个双端队列的实例，D应该支持四个基本操作：

- D.add_first(e): 在双端队列Q的头部添加一个元素e。
- D.add_last(e): 在双端队列Q的尾部添加一个元素e。
- D.del_first()：如果双端队列D中至少含有一个元素，从队列Q的头部移除一个元素并返回其值；如果队列Q为空，则抛出异常。
- D.del_last()：如果双端队列D中至少含有一个元素，从队列Q的尾部移除一个元素并返回其值；如果队列Q为空，则抛出异常。

此外，为了方便操作，栈S还应支持：

- D.first()：在不移除双端队列D头部元素的情况下返回元素的值；如果双端队列D为空，则抛出异常。
- D.last()：在不移除双端队列D尾部元素的情况下返回元素的值；如果双端队列D为空，则抛出异常。
- D.is_empty(): 判断双端队列D是否为空，如果为空就返回True；反之返回False。
- len(D): 返回双端队列D中元素的个数。
- iter(Q): 返回一个Iterator。

In [ ]:
from abc import ABC, abstractmethod


class AbstractDeque(ABC):

    def __init__(self):
        self._size = 0

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    @abstractmethod
    def __iter__(self):
        raise NotImplementedError    
    
    @abstractmethod
    def add_first(self, e):
        raise NotImplementedError

    @abstractmethod
    def add_last(self, e):
        raise NotImplementedError

    @abstractmethod
    def del_first(self):
        raise NotImplementedError

    @abstractmethod
    def del_last(self):
        raise NotImplementedError

    @abstractmethod
    def first(self):
        raise NotImplementedError

    @abstractmethod
    def last(self):
        raise NotImplementedError


In [ ]:
class ArrayDeque(AbstractDeque):
    def __init__(self, data=[]):
        super().__init__()
        self._array = data
        self._size = len(data)
        
    def __iter__(self):
        return (x for x in self._array)

    def add_first(self, e):
        self._array.insert(0, e)
        self._size += 1

    def add_last(self, e):
        self._array.append(e)
        self._size += 1

    def del_first(self):
        if self.is_empty():
            raise IndexError("Deque is empty")
        self._array.pop(0)
        self._size -= 1

    def del_last(self):
        if self.is_empty():
            raise IndexError("Deque is empty")
        self._array.pop()
        self._size -= 1

    def first(self):
        if self.is_empty():
            raise IndexError("Deque is empty")
        return self._array[0]

    def last(self):
        if self.is_empty():
            raise IndexError("Deque is empty")
        return self._array[-1]


In [ ]:
l1 = [1, 2, 3]
deque1 = ArrayDeque(l1)
deque1.del_first()
deque1.del_last()
deque1.add_first(0)
deque1.add_last(4)
deque1._array

上面的ArrayDeque在构造时有个bug大家发现了吗？这个bug会导致什么问题？如何正确的书写？

In [ ]:
l1

In [ ]:
deque2 = ArrayDeque()
deque2.add_first('a')
deque2.add_last('b')
deque2._array

In [ ]:
deque3 = ArrayDeque()
deque3._array

In [ ]:
deque3.add_first('What')
deque3.add_last('TF')
deque2._array

In [ ]:
deque2._array is deque3._array

注意：在Python中如果使用mutable类型作为函数参数时，默认值要使用None！

正确的书写：

```python
def __init__(self, data=None):
    super().__init__()
    if data is None:
        self._array = []
    else:
        self._array = list(data)
    self._size = len(self._array)
```

#### 循环双端队列（Circular Deque)

- 与Circular Queue类似
- 课后作业

### collections.deque

[Python3官方文档](https://docs.python.org/3/library/collections.html#collections.deque)

> Deques are a generalization of stacks and queues (the name is pronounced “deck” and is short for “double-ended queue”). Deques support thread-safe, memory efficient appends and pops from either side of the deque with approximately the same O(1) performance in either direction.

In [ ]:
import time
from collections import deque

num = 100_000

def append(c):
    if isinstance(c, deque):
        for i in range(num):
            c.append(i)
    else:
        for i in range(num):
            c.add_last(i)
            
def appendleft(c):
    if isinstance(c, deque):
        for i in range(num):
            c.appendleft(i)
    else:
        for i in range(num):
            c.add_first(i)
def pop(c):
    if isinstance(c, deque):
        for i in range(num):
            c.pop()
    else:
        for i in range(num):
            c.del_last()

def popleft(c):
    if isinstance(c, deque):
        for i in range(num):
            c.popleft()
    else:
        for i in range(num):
            c.del_first()

for container in [deque, ArrayDeque]:
    for operation in [append, appendleft, pop, popleft]:
        c = container([x for x in range(num)])
        start = time.perf_counter()
        operation(c)
        elapsed = time.perf_counter() - start
        print("Completed %s/%s in %.2f seconds: %.1f ops/sec" % (container.__name__, operation.__name__, elapsed, num / elapsed))

In [ ]:
from collections import deque

def palchecker(aString):
    chardeque = deque()

    for ch in aString:
        chardeque.append(ch)

    stillEqual = True
    while len(chardeque) > 1 and stillEqual:
        first = chardeque.popleft()
        last = chardeque.pop()
        if first != last:
            stillEqual = False

    return stillEqual

print(palchecker("lsdkjfskf"))
print(palchecker("radar"))

- collections.deque底层使用了双向链表，所以在头尾插入和删除数据的时间复杂度可以达到O(1)。
- 如果需要操作中间索引的数据，建议使用List。

# Any Questions?

## 实践项目

- 期中考试还是实践项目？
- 如何选择实践课题？
- 如何组织项目的开发？

- 期中考试 -> 实践项目 (20%)
- 以下是可以用来实践的课题，但更希望大家能提出自己的点子，实现自己的想法：
    - 解析Verilog，DEF...
    - 抽取EDA工具的log信息
    - 分析PD项目中的数据
    - IT监控，比如磁盘监控、机器监控、LSF任务的监控
    - ......
- 开源项目，每个人都能做出贡献（不仅仅限于代码的提交）

## 课后作业 Assignment-04

1) 已知有两个序列pushed和popped分别表示了对栈的push和pop的元素顺序，用Python实现一个函数validateStackSequences(pushed, popped):确定这个两个对栈的操作序列能否可以在一个空栈上操作成功。

```python
def validateStackSequences(self, pushed, popped):
    """Return True if and only if this sequence of pushed and popped operations can been operated on an empty stack successfully.
    :type pushed: List[int]
    :type popped: List[int]
    :rtype: bool
    """
    pass
```


>例子1：  
>输入: pushed = [1, 2, 3, 4, 5], popped = [4, 5, 3, 2, 1]  
>输出: True  
>解释: 可以按照以下顺利对一个空栈进行操作 —— push(1), push(2), push(3), push(4), pop() -> 4,
push(5), pop() -> 5, pop() -> 3, pop() -> 2, pop() -> 1

>例子2:  
输入: pushed = [1, 2, 3, 4, 5], popped = [4, 3, 5, 1, 2]  
输出: False  
解释: 在popped指定的操作顺序下，元素1不能先于元素2执行弹出操作。

注：
- 0 <= len(pushed) == len(popped) <= 1000
- 0 <= pushed[i], popped[i] < 1000
- popped序列可以看做是pushed序列的一个重排列，
- pushed序列中的值不重复，popped序列中的值也不重复。

2) 用Python实现一个算法，该算法可以找出一个序列中每个元素后续出现的第一个比它大的元素，并记录这个比它大的元素的索引。如果在右边找不到比此元素更大的值则在输出的相对应位置上记录-1。

```python
def findRightFirstLargeNum(l):
    """
    :type l: List[int]
    :rtype: List[int]
    """
```

>例子1：  
输入：[2, 6, 0, 1, 7, 3, 5, 8, 4, 9]  
输出：[1, 4, 3, 4, 7, 6, 7, 9, 9, -1]  
解释：对于输入中的第一个元素“2”来说，其右边第一个比它大的数是“6”，所以输出的结果中第一个元素应该记录“6”的索引就是“1”；对于输入中第二个元素“6”来说，其右边第一个比它大的数是“7”，所以输出结果中第二个元素应该记录“7”所在的索引是“4”；依次类推。。。

>例子2:  
输入：[7, 6, 3, 5, 8, 1, 9, 4, 0, 2]  
输出：[4, 4, 3, 4, 6, 6, -1, -1, 9, -1] 

3）参考课件中Circular Queue环形队列的代码，自己实现一个Circular Deque环形双端队列。
注：可以不必继承课件中的Deque ADT，而直接实现Circular deque，但是必须包括以下的基本功能，

- add_first(e): 如果deque还有空位，就在队列的头部插入一个新元素e。
- add_last(e): 如果deque还有空位，就在队列的尾部插入一个新元素e。
- del_first(): 如果deque不为空，就删除队列头部的第一个元素。
- del_last(): 如果deque不为空，就删除队列尾部的第一个元素。
- first(): 如果deque不为空，则返回其头部的第一个元素的值，但并不删除此元素。
- last(): 如果deque不为空，则返回其尾部的第一个元素的值，但并不删除此元素。